In [2]:
import pandas as pd
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
import string

In [3]:
sentences = pd.read_csv('texts.csv')
words_df = pd.read_csv('words.csv')
words_df[['nmb_text', 'nmb_sent', 'nmb_word']] = words_df[['nmb_text', 'nmb_sent', 'nmb_word']].astype(int)
sentences[['nmb_text', 'nmb_sent']] = sentences[['nmb_text', 'nmb_sent']].astype(int)

In [6]:
def search_func(search_first):
    words_df[['nmb_text', 'nmb_sent', 'nmb_word']] = words_df[['nmb_text', 'nmb_sent', 'nmb_word']].astype(int)
    sentences[['nmb_text', 'nmb_sent']] = sentences[['nmb_text', 'nmb_sent']].astype(int)

    if len(search_first)==1 and search_first in list(string.punctuation):
        return 'wrong_query'
    if search_first == '':
        return 'wrong_query'
    for i in search_first.split():
        if i in list(string.punctuation):
            return 'wrong_query'
    for i in search_first:
        if i in list(string.punctuation) and i!='"' and i!='+' and i!="'" and i!="-":
            return 'wrong_query'
            
    if search_first[0]!='"' and search_first[-1]!='"':
        res = []
        search = search_first.split()
        text_id = 1
        while text_id <= max(words_df['nmb_text']):
            df = words_df[words_df['nmb_text'] == text_id]
            sent_ind = 1
            all = []
            spis = {}
            while sent_ind <= max(df['nmb_text']):
                srch_i = 0
                words = df[df['nmb_sent'] == sent_ind]['word'].tolist()
                tags = df[df['nmb_sent'] == sent_ind]['pos'].tolist()
                lemmas = df[df['nmb_sent'] == sent_ind]['lemma'].tolist()
                word_i = 0
                spis_n = []
                while word_i<len(words) and srch_i<len(search):
                    if search[srch_i][0]=='"' and search[srch_i][-1]=='"':
                        if '+' in search[srch_i]:
                            d = search[srch_i].split('+')
                            if d[0]==words[word_i] and d[1]==tags[word_i]:
                                spis_n.append(words[word_i])
                                srch_i+=1
                            else:
                                word_i -=len(spis_n)
                                srch_i = 0
                                spis_n = []
                        elif words[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        elif tags[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        else:
                            word_i -=len(spis_n)
                            srch_i = 0
                            spis_n = []
                    elif search[srch_i][0]!='"' and search[srch_i][-1]!='"':
                        lem = m.parse(search[srch_i])[0].normal_form
                        if '+' in search[srch_i]:
                            d = search[srch_i].split('+')
                            if m.parse(d[0])[0].normal_form==lemmas[word_i] and d[1]==tags[word_i]:
                                spis_n.append(words[word_i])
                                srch_i+=1
                            else:
                                word_i -=len(spis_n)
                                srch_i = 0
                                spis_n = []
                        elif lemmas[word_i] == lem:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        elif tags[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        else:
                            word_i -=len(spis_n)
                            srch_i = 0
                            spis_n = []
                    if spis_n!=[] and len(search) == len(spis_n):
                        if sent_ind not in spis.keys():
                            spis[sent_ind] = []
                            spis[sent_ind].append(' '.join(spis_n))
                        else:
                            spis[sent_ind].append(' '.join(spis_n))
                        srch_i = 0
                        word_i -=len(spis_n)
                        word_i += 1
                        spis_n = []
                    word_i+=1
                sent_ind+=1   
            all = [[spis[k], sentences[(sentences['nmb_text'] == text_id) & (sentences['nmb_sent'] == k)]['sentence'].tolist()[0]] for k in spis.keys()]
            if all != []:
                res.append([all, sentences[sentences['nmb_text'] == text_id]['author'].tolist()[0], sentences[sentences['nmb_text'] == text_id]['title'].tolist()[0]])
            text_id+=1
            spis = {}

    elif search_first[0]=='"' and search_first[-1]=='"':
        res = []
        search = (search_first[1:-1]).split()
        text_id = 1
        while text_id <= max(words_df['nmb_text']):
            df = words_df[words_df['nmb_text'] == text_id]
            sent_ind = 0
            all = []
            spis = {}
            while sent_ind <= max(df['nmb_text']):
                srch_i = 0
                words = df[df['nmb_sent'] == sent_ind]['word'].tolist()
                tags = df[df['nmb_sent'] == sent_ind]['pos'].tolist()
                lemmas = df[df['nmb_sent'] == sent_ind]['lemma'].tolist()
                word_i = 0
                spis_n = []
                while word_i<len(words) and srch_i<len(search):
                    if '+' in search[srch_i]:
                        dat = search[srch_i].split('+')
                        if dat[0]==words[word_i] and dat[1]==tags[word_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        else:
                            word_i -=len(spis_n)
                            srch_i = 0
                            spis_n = []
                    elif words[word_i] == search[srch_i]:
                        spis_n.append(words[word_i])
                        srch_i+=1
                    elif tags[word_i] == search[srch_i]:
                        spis_n.append(words[word_i])
                        srch_i+=1
                    else:
                        word_i -=len(spis_n)
                        srch_i = 0
                        spis_n = []
                    if spis_n!=[] and len(search) == len(spis_n):
                        if sent_ind not in spis.keys():
                            spis[sent_ind] = []
                            spis[sent_ind].append(' '.join(spis_n))
                        else:
                            spis[sent_ind].append(' '.join(spis_n))
                        srch_i = 0
                        word_i -=len(spis_n)
                        word_i += 1
                        spis_n = []
                    word_i+=1
                sent_ind+=1
            all = [[spis[k], sentences[(sentences['nmb_text'] == text_id) & (sentences['nmb_sent'] == k)]['sentence'].tolist()[0]] for k in spis.keys()]
            if all != []:
                res.append([all, sentences[sentences['nmb_text'] == text_id]['author'].tolist()[0], sentences[sentences['nmb_text'] == text_id]['title'].tolist()[0]])
            text_id+=1
            spis = {}
    
    elif (search_first[0]=='"' and search_first[-1]!='"') or (search_first[0]!='"' and search_first[-1]=='"'):
        res = []
        search = search_first.split()
        text_id = 1
        while text_id <= max(words_df['nmb_text']):
            df = words_df[words_df['nmb_text'] == text_id]
            sent_ind = 1
            all = []
            spis = {}
            while sent_ind <= max(df['nmb_text']):
                srch_i = 0
                words = df[df['nmb_sent'] == sent_ind]['word'].tolist()
                tags = df[df['nmb_sent'] == sent_ind]['pos'].tolist()
                lemmas = df[df['nmb_sent'] == sent_ind]['lemma'].tolist()
                word_i = 0
                spis_n = []
                while word_i<len(words) and srch_i<len(search):
                    if search[srch_i][0]=='"' and search[srch_i][-1]=='"':
                        if '+' in search[srch_i]:
                            d = search[srch_i].split('+')
                            if d[0]==words[word_i] and d[1]==tags[word_i]:
                                spis_n.append(words[word_i])
                                srch_i+=1
                            else:
                                word_i -=len(spis_n)
                                srch_i = 0
                                spis_n = []
                        elif words[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        elif tags[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        else:
                            word_i -=len(spis_n)
                            srch_i = 0
                            spis_n = []
                    elif search[srch_i][0]!='"' and search[srch_i][-1]!='"':
                        lem = m.parse(search[srch_i])[0].normal_form
                        if '+' in search[srch_i]:
                            d = search[srch_i].split('+')
                            if m.parse(d[0])[0].normal_form==lemmas[word_i] and d[1]==tags[word_i]:
                                spis_n.append(words[word_i])
                                srch_i+=1
                            else:
                                word_i -=len(spis_n)
                                srch_i = 0
                                spis_n = []
                        elif lemmas[word_i] == lem:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        elif tags[word_i] == search[srch_i]:
                            spis_n.append(words[word_i])
                            srch_i+=1
                        else:
                            word_i -=len(spis_n)
                            srch_i = 0
                            spis_n = []
                    if spis_n!=[] and len(search) == len(spis_n):
                        if sent_ind not in spis.keys():
                            spis[sent_ind] = []
                            spis[sent_ind].append(' '.join(spis_n))
                        else:
                            spis[sent_ind].append(' '.join(spis_n))
                        srch_i = 0
                        word_i -=len(spis_n)
                        word_i += 1
                        spis_n = []
                    word_i+=1
                sent_ind+=1   
            all = [[spis[k], sentences[(sentences['nmb_text'] == text_id) & (sentences['nmb_sent'] == k)]['sentence'].tolist()[0]] for k in spis.keys()]
            if all != []:
                res.append([all, sentences[sentences['nmb_text'] == text_id]['author'].tolist()[0], sentences[sentences['nmb_text'] == text_id]['title'].tolist()[0]])
            text_id+=1
            spis = {}

    return res

In [9]:
#ваш запрос
search = 'NOUN'
search_func(search)

[[[[['поучений',
     'мужчина',
     'дурак',
     'женщина',
     'дура',
     'марк',
     'твен',
     'письма',
     'земли'],
    '       "Из прежних моих поучений      вы могли вывести, что      мужчина - дурак; теперь вам известно,      что женщина - дьявольская дура"      Марк Твен      "Письма с земли.']],
  'Чингиз Абдуллаев',
  'Измена в имени Твоем'],
 [[[['место', 'земле'],
    '         "Оставьте, пожалуйста, это место таким же  чистым, каким  его нашли..." Но это же  относится ко  всей Земле, а  не только к уборным!']],
  'Чингиз Абдуллаев',
  'Сотвори себе мир'],
 [[[['эпиграф', 'участникам', 'войны', 'афганистане', 'книгу', 'часть'],
    '        [Эпиграф]      ВСЕМ УЧАСТНИКАМ  ВОЙНЫ В АФГАНИСТАНЕ, обманутым и забытым,  преданным и брошенным, посвящаю эту книгу     * ЧАСТЬ I.'],
   [['эпиграф', 'мир', 'подробностях', 'подробностей', 'множество', 'знания'],
    '"Уйти..." *       [Эпиграф]      "Чтобы   постичь  окружающий  нас   мир,  нужно   знать   его  во  всех под